In [ ]:
# Script unifié et corrigé pour créer un Google Sheet de calcul fiscal.
# Exécutez ce script dans un environnement Google Colab.

from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
import google.auth
import pandas as pd
import numpy as np

def create_tax_calculator_sheet():
    """Crée et remplit la feuille de calcul Google Sheets pour le calcul de l'impôt."""
    # --- Authentification et création de la feuille de calcul ---
    try:
        auth.authenticate_user()
        credentials, project = google.auth.default()
        gc = gspread.authorize(credentials)
        sh = gc.create('Tax_123_bis_CGI_corrige')
    except Exception as e:
        print(f"Erreur lors de l'authentification ou de la création de la feuille : {e}")
        return

    # --- Fonction utilitaire pour écrire des DataFrames ---
    def write_df_to_sheet(df, sheet_name):
        try:
            worksheet = sh.add_worksheet(title=sheet_name, rows=1, cols=1)
        except gspread.exceptions.APIError:
            worksheet = sh.worksheet(sheet_name)
        worksheet.clear()
        set_with_dataframe(worksheet, df, include_index=False, allow_formulas=True)
        print(f"Feuille '{sheet_name}' créée et mise à jour.")

    # --- Définition et écriture des feuilles de paramètres ---
    # Feuille: Baremes_IR
    baremes_data = {
        'Année': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
        'Seuil1': [9710, 9807, 9964, 10064, 10084, 10225, 10777, 11294, 11497, 11704],
        'Taux1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'Seuil2': [26818, 27086, 27519, 27794, 25710, 26070, 27478, 28797, 29315, 29843],
        'Taux2': [0.14, 0.14, 0.14, 0.14, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11],
        'Seuil3': [71898, 72617, 73779, 74517, 73516, 74545, 78570, 82341, 83823, 85332],
        'Taux3': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
        'Seuil4': [152260, 153783, 156244, 157806, 158122, 160336, 168994, 177106, 180294, 183539],
        'Taux4': [0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41],
        'Seuil5': [1000000000] * 10,
        'Taux5': [0.45] * 10
    }
    df_baremes = pd.DataFrame(baremes_data)
    write_df_to_sheet(df_baremes, 'Baremes_IR')

    # Feuille: Param_IR
    param_ir_data = {
        'Année': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
        'Decote_A_celib': [1165, 1177, 1196, 1208, 779, 779, 833, 833, 889, 905],
        'Decote_B_celib': [0.75, 0.75, 0.75, 0.75, 0.4525, 0.4525, 0.4525, 0.4525, 0.4525, 0.4525],
        'Decote_plafond_celib': [1553, 1569, 1595, 1611, 1722, 1722, 1841, 1841, 1965, 2000],
        'Decote_A_couple': [1920, 1939, 1970, 1990, 1289, 1289, 1378, 1378, 1470, 1497],
        'Decote_B_couple': [0.75, 0.75, 0.75, 0.75, 0.4525, 0.4525, 0.4525, 0.4525, 0.4525, 0.4525],
        'Decote_plafond_couple': [2560, 2585, 2627, 2653, 2849, 2849, 3045, 3045, 3249, 3308],
        'CEHR_seuil1_celib': [250000] * 10, 'CEHR_seuil2_celib': [500000] * 10,
        'CEHR_taux1': [0.03] * 10, 'CEHR_taux2': [0.04] * 10,
        'CEHR_seuil1_couple': [500000] * 10, 'CEHR_seuil2_couple': [1000000] * 10,
        'QF_cap_par_demi_part (€)': [1512, 1527, 1552, 1570, 1552, 1570, 1592, 1678, 1751, 1783]
    }
    df_param_ir = pd.DataFrame(param_ir_data)
    write_df_to_sheet(df_param_ir, 'Param_IR')

    # Feuille: Param_IS (CORRIGÉ - Ajout de la feuille manquante)
    param_is_data = {
        'Année': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
        'Plafond_taux_reduit_15_pourcent': [38120, 38120, 38120, 38120, 38120, 38120, 42500, 42500, 42500, 42500]
    }
    df_param_is = pd.DataFrame(param_is_data)
    write_df_to_sheet(df_param_is, 'Param_IS')

    # --- Feuille de saisie des variables ---
    variables_data = {
        'Année': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
        'Parts fiscales': [2.0] * 10, 'Revenu net imposable déclaré (€)': [100000] * 10,
        'Revenu réputé distribué (123 bis CGI) (€)': [50000] * 10,
        'Majoration 25 % (VRAI/FAUX)': [True] * 10, 'Option PFU (VRAI/FAUX)': [False] * 10,
    }
    df_variables = pd.DataFrame(variables_data)
    write_df_to_sheet(df_variables, 'Variables_a_renseigner')

    # --- Feuille de calcul IS (avec formules CORRIGÉES) ---
    calcul_is_columns = ['Année', 'Bénéfice imposable avant ajustements (€)', 'Dividendes perçus (€)', 'Régime mère-fille applicable (VRAI/FAUX)', 'Bénéfice imposable ajusté (€)', 'Chiffre d\'affaires (CA) (€)', 'PME éligible taux 15 % (VRAI/FAUX)', 'IS théorique total (€)']
    df_calcul_is = pd.DataFrame(np.nan, index=range(10), columns=calcul_is_columns)
    df_calcul_is['Année'] = [y for y in range(2016, 2026)]

    calcul_is_formulas = {
        'Bénéfice imposable ajusté (€)': "=IF(D{row}, B{row}-C{row}+0.05*C{row}, B{row})",
        'IS théorique total (€)': """=MIN(E{row}, VLOOKUP(A{row},'Param_IS'!$A:$B,2,FALSE))*0.15 + MAX(0, E{row}-VLOOKUP(A{row},'Param_IS'!$A:$B,2,FALSE)) * IF(A{row}>=2022, 0.25, IF(A{row}=2021, 0.265, IF(A{row}=2020, 0.28, IF(A{row}=2019, 0.31, 0.3333))))"""
    }
    for col, formula_template in calcul_is_formulas.items():
        df_calcul_is[col] = [formula_template.format(row=i+2) for i in range(len(df_calcul_is))]
    write_df_to_sheet(df_calcul_is, 'Calcul_IS')

    print(f"\nFeuille de calcul créée avec succès : https://docs.google.com/spreadsheets/d/{sh.id}")
    # Nettoyage de la feuille par défaut
    try:
        default_sheet = sh.worksheet('Sheet1')
        sh.del_worksheet(default_sheet)
    except gspread.WorksheetNotFound:
        pass

# --- Exécution du script ---
create_tax_calculator_sheet()
